## Day 47 Lecture 1 Assignment

In this assignment, we will apply k-means clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [ ]:
# answer goes here
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv')
df.head(20)



,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,0.357,1.3,4.1,0.323,0.5,1.0,0.500,0.487,0.4,0.4,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,0.222,0.2,1.5,0.133,0.2,0.3,0.667,0.278,0.7,1.0,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,0.345,0.7,2.2,0.338,0.4,1.1,0.361,0.459,0.2,0.3,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
5,2019,Deng Adel,SF,21,CLE,19,3,10.2,0.6,1.9,0.306,0.3,1.2,0.261,0.3,0.7,0.385,0.389,0.2,0.2,1.000,0.2,0.8,1.0,0.3,0.1,0.2,0.3,0.7,1.7
6,2019,DeVaughn Akoon-Purcell,SG,25,DEN,7,0,3.1,0.4,1.4,0.300,0.0,0.6,0.000,0.4,0.9,0.500,0.300,0.1,0.3,0.500,0.1,0.4,0.6,0.9,0.3,0.0,0.3,0.6,1.0
7,2019,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,0.519,0.1,0.5,0.238,8.3,15.8,0.528,0.522,4.3,5.1,0.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
8,2019,Rawle Alkins,SG,21,CHI,10,1,12.0,1.3,3.9,0.333,0.3,1.2,0.250,1.0,2.7,0.370,0.372,0.8,1.2,0.667,1.1,1.5,2.6,1.3,0.1,0.0,0.8,0.7,3.7
9,2019,Grayson Allen,SG,23,UTA,38,2,10.9,1.8,4.7,0.376,0.8,2.6,0.323,0.9,2.1,0.443,0.466,1.2,1.6,0.750,0.1,0.5,0.6,0.7,0.2,0.2,0.9,1.2,5.6


The goal is to cluster these player-seasons to identify potential player "archetypes".

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [ ]:
# answer goes here
df_clean = df[df.GS>=20]
df_clean = df_clean[df_clean.MP>=10]
df_clean.head(20)



,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
7,2019,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,0.519,0.1,0.5,0.238,8.3,15.8,0.528,0.522,4.3,5.1,0.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
10,2019,Jarrett Allen,C,20,BRK,80,80,26.2,4.2,7.1,0.590,0.1,0.6,0.133,4.1,6.5,0.629,0.595,2.5,3.5,0.709,2.4,6.0,8.4,1.4,0.5,1.5,1.3,2.3,10.9
12,2019,Al-Farouq Aminu,PF,28,POR,81,81,28.3,3.2,7.3,0.433,1.2,3.5,0.343,2.0,3.9,0.514,0.514,1.9,2.1,0.867,1.4,6.1,7.5,1.3,0.8,0.4,0.9,1.8,9.4
14,2019,Kyle Anderson,SF,25,MEM,43,40,29.8,3.5,6.4,0.543,0.2,0.8,0.265,3.3,5.6,0.583,0.560,0.9,1.5,0.578,1.1,4.7,5.8,3.0,1.3,0.9,1.3,2.6,8.0
17,2019,Giannis Antetokounmpo,PF,24,MIL,72,72,32.8,10.0,17.3,0.578,0.7,2.8,0.256,9.3,14.5,0.641,0.599,6.9,9.5,0.729,2.2,10.3,12.5,5.9,1.3,1.5,3.7,3.2,27.7
21,2019,Ryan Arcidiacono,PG,24,CHI,81,32,24.2,2.3,5.2,0.447,1.0,2.7,0.373,1.3,2.5,0.527,0.544,1.1,1.3,0.873,0.3,2.4,2.7,3.3,0.8,0.0,0.8,2.1,6.7
22,2019,Trevor Ariza,SF,33,TOT,69,69,34.0,4.3,10.7,0.399,2.1,6.3,0.334,2.2,4.4,0.493,0.498,1.9,2.4,0.793,0.7,4.7,5.4,3.7,1.3,0.3,1.5,1.9,12.5
23,2019,D.J. Augustin,PG,31,ORL,81,81,28.0,3.9,8.4,0.470,1.6,3.8,0.421,2.3,4.5,0.511,0.566,2.2,2.6,0.866,0.5,2.0,2.5,5.3,0.6,0.0,1.6,1.4,11.7


Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [ ]:
# answer goes here
data = df_clean[['2P%', '3P%', 'FG%', 'FT%', 'PTS']]
data



,2P%,3P%,FG%,FT%,PTS
3,0.596,0.000,0.595,0.500,13.9
4,0.588,0.200,0.576,0.735,8.9
7,0.528,0.238,0.519,0.847,21.3
10,0.629,0.133,0.590,0.709,10.9
12,0.514,0.343,0.433,0.867,9.4
...,...,...,...,...,...
2122,0.454,0.344,0.414,0.869,14.1
2125,0.464,0.344,0.408,0.830,15.3
2126,0.504,0.402,0.452,0.833,11.7
2138,0.523,0.233,0.514,0.644,15.1


Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [ ]:
# answer goes here
scaler = StandardScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
data_scaled = data_scaled.dropna()
data_scaled




,2P%,3P%,FG%,FT%,PTS
0,1.596164,-2.951968,2.168189,-2.586447,0.114638
1,1.454230,-1.138289,1.849645,-0.255221,-0.744985
2,0.389723,-0.793690,0.894013,0.855832,1.386880
3,2.181642,-1.745872,2.084361,-0.513144,-0.401136
4,0.141338,0.158491,-0.547818,1.054234,-0.659023
...,...,...,...,...,...
799,-0.923168,0.167559,-0.866362,1.074074,0.149023
800,-0.745750,0.167559,-0.966955,0.687190,0.355332
801,-0.036079,0.693526,-0.229274,0.716950,-0.263596
802,0.301014,-0.839032,0.810185,-1.157951,0.320947


Run K-means clustering with K = 3 and print out the resulting centroids. When printing the centroids, transform the scaled centroids back into their corresponding unscaled values. What "archetypes" do you see?

In [ ]:
# answer goes here
kmeans = KMeans(n_clusters=3)
data_scaled['cluster'] = kmeans.fit_predict(data_scaled)
centers = pd.DataFrame(kmeans.cluster_centers_, columns=data_scaled.columns)
centers_scaled = pd.DataFrame(scaler.inverse_transform(centers.drop('cluster', axis=1)), columns=data.columns)
centers_scaled





,2P%,3P%,FG%,FT%,PTS
0,0.472977,0.348330,0.427236,0.768179,10.366494
1,0.516456,0.367393,0.472710,0.816276,19.204779
2,0.569325,0.148596,0.543544,0.639395,9.939474


In [ ]:
#There seem to be high 2point scorers, high point scorers and low 3 point scorers

Experiment with different values of K. Do any further interesting archetypes come out?

In [ ]:
# answer goes here
kmeans = KMeans(n_clusters=6)
data_scaled['cluster'] = kmeans.fit_predict(data_scaled)
centers = pd.DataFrame(kmeans.cluster_centers_, columns=data_scaled.columns)
centers_scaled = pd.DataFrame(scaler.inverse_transform(centers.drop('cluster', axis=1)), columns=data.columns)
centers_scaled




,2P%,3P%,FG%,FT%,PTS
0,0.487951,0.331380,0.438485,0.704988,9.088344
1,0.583404,0.317135,0.533096,0.648250,9.311538
2,0.503598,0.357349,0.462213,0.827870,22.012426
3,0.558594,0.030453,0.553563,0.629297,10.418750
4,0.536483,0.378567,0.486817,0.798617,13.865833
5,0.455005,0.358143,0.413167,0.817862,11.485222


In [ ]:
#There seems to be a greater division in point scorers than other groups